## Ques3: Implement Linear Discriminant Analysis (LDA) step-by-step on Iris dataset (present in sklearn.datasets).

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris

In [2]:
iris=load_iris()
X=iris.data
y=iris.target

### Between-class scatter matrix Sb

In [3]:
N=np.bincount(y) # number of samples for given class
mean_vecs=[]
[mean_vecs.append(np.mean(X[y==i],axis=0)) for i in range(3)] # class means
mean_overall = np.mean(X, axis=0) # overall mean
S_B=np.zeros((4,4))
for i in range(3):
    S_B += N[i]*np.dot((mean_vecs[i]-mean_overall).reshape(4,1), (mean_vecs[i]-mean_overall).reshape(1,4))

In [4]:
S_B

array([[ 63.21213333, -19.95266667, 165.2484    ,  71.27933333],
       [-19.95266667,  11.34493333, -57.2396    , -22.93266667],
       [165.2484    , -57.2396    , 437.1028    , 186.774     ],
       [ 71.27933333, -22.93266667, 186.774     ,  80.41333333]])

### Within-class scatter matrix Sw

In [5]:
S_W = np.zeros((4,4))
for i,mv in enumerate(mean_vecs):
    class_sc_mat = np.zeros((4,4))                  # scatter matrix for every class
    for row in X[y==i]:
        row, mv = row.reshape(4,1), mv.reshape(4,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat                             # sum class scatter matrices


In [6]:
S_W

array([[38.9562, 13.63  , 24.6246,  5.645 ],
       [13.63  , 16.962 ,  8.1208,  4.8084],
       [24.6246,  8.1208, 27.2226,  6.2718],
       [ 5.645 ,  4.8084,  6.2718,  6.1566]])

### Compute the eigen values and vectors of the scatter matrix

In [7]:
eigen_vals, eigen_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

### Sort the eigenvectors by decreasing eigenvalues

In [8]:
eigen_pairs = []
for i in range(len(eigen_vals)):
    eigen_pairs.append([eigen_vals[i], eigen_vecs[i]])
eigen_pairs = sorted(eigen_pairs, reverse=True)

In [9]:
eigen_pairs

[[32.19192919827803,
  array([ 0.20874182, -0.00653196,  0.88513899, -0.80593687])],
 [0.28539104262306414,
  array([ 0.38620369, -0.58661055, -0.29455053,  0.40432808])],
 [3.5296362660244315e-15,
  array([-0.55401172,  0.25256154, -0.27255052,  0.41273963])],
 [3.17116800810927e-17,
  array([-0.7073504 , -0.76945309, -0.23555291, -0.12895956])]]

In [10]:
eigv_sum = sum(eigen_vals)
for i,j in enumerate(eigen_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i, (j[0]/eigv_sum)))

eigenvalue 0: 99.12%
eigenvalue 1: 0.88%
eigenvalue 2: 0.00%
eigenvalue 3: 0.00%


### Taking most variance explained first 2 eigen vectors

In [11]:
W=np.hstack((eigen_pairs[0][1][:, ].reshape(4,1),eigen_pairs[1][1][:, ].reshape(4,1)))
X_lda = X.dot(W) # Y = X*W

In [12]:
data=pd.DataFrame(X_lda)
data['class']=y
data.columns=["LD1","LD2","class"]
data

,LD1,LD2,class
0,2.119729,-0.415003,0
1,2.081246,-0.198939,0
2,1.949678,-0.364047,0
3,2.106484,-0.402916,0
4,2.098201,-0.512285,0
...,...,...,...
145,4.128042,0.226025,2
146,4.193158,0.262028,2
147,4.328075,0.027486,2
148,4.198086,-0.260631,2
